In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

url = "C:/Workspace/StarWarsClassifier/troop_movements.csv"
data = pd.read_csv(url)

df = pd.DataFrame(data)

X = df[['homeworld', 'unit_type']]
# y = df['is_resistance']
print(df)

# X_encoded = pd.get_dummies(X)

# X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# model = DecisionTreeClassifier()
# model.fit(X_train, y_train)
# pred = model.predict(X_test)

# accuracy = accuracy_score(y_test, pred)
# print(accuracy)

# importances = model.feature_importances_
# feature_importances = pd.DataFrame({'Feature': X_encoded.columns, 'Importance': importances})